In [ ]:
import json
import re

with open('./detailed-lcc.json', 'r') as reader:
    detail = json.load(reader)

with open('./top-level-lcc.json', 'r') as reader:
    basic = json.load(reader)

In [ ]:
ex1 = 'HV-1568.00000000.B376 2016' #The Minority Body by Elizabeth Barnes
ex2 = 'DAW1008.00000000.B37 1987' #A guide to Central Europe by Bassett, Richard
ex3 = 'E--185.86.00000000.H739 2001' #Salvation: Black People and Love by bell hooks
ex4 = 'PL-0788.40000000.G4 E5 2000' #Genji Monogatari by Murasaki Shikibu
ex5 = 'DS-0032.80000000' #Orientalism by Edward Said

In [ ]:
def split_lcc(call_number): #returns tuple
    out = re.findall(r'[A-Z]+-*\d+\.*[1-9]+', call_number)[0] #specifically for how OL formats their LCC call numbers
    out = out.replace('-', '')
    out = re.sub(r'(?<=[A-Z])0+', '', out)

    match = re.match(r'([A-Z]+)(\d+\.*\d+)', out)
    if match:
        return (match.group(1), float(match.group(2)))

def lookup_meaning(code): #takes in tuple (call number)
    l = []

    try:  
        l.append(basic[code[0]][0]['subject']) #get overarching subject

    except:
        pass

    try:
        d = detail[code[0]]
        for i in d:
            if code[1] >= i['start'] and code[1] <= i['stop']:
                l.append(i['subject'])
    except:
        pass

    return l
    #returns a list of definitions for the code

In [ ]:
print(lookup_meaning(split_lcc(ex1)))
print(lookup_meaning(split_lcc(ex2)))
print(lookup_meaning(split_lcc(ex3)))
print(lookup_meaning(split_lcc(ex4)))
print(lookup_meaning(split_lcc(ex5)))

In [2]:
#function to search by lccn. fields is a string of comma-seperated fields to return
def search_lccn(lccn, fields = 'author_name,subject,lcc,title', limit = 5): 
    r = []
    response = requests.get(f'https://openlibrary.org/search.json?q=lcc:{lccn}*&fields={fields}&limit={limit}').json()
    return response['docs']
    
    #return r

In [ ]:
#this could be made simpler; right now i return a tuple for simplification when looking up keys in the dictionary
search_lccn(split_lcc(ex2)[0] + str(split_lcc(ex2)[1]), 'subject', 1)

In [ ]:
#queries
#1. Direct call number: HV3181
print(search_lccn('HV3181', 'title, lcc', 1))

#2. Anything under a broad category: H* includes H, HA, HB, etc.
print(search_lccn('H', 'title', 1))

#3. Anything under a specific category: H-- includes only H, HA- includes only HA
print(search_lccn('H--', 'title', 1))
#books might differ slightly in exact code, so i limit to only title here, you can look at the lcc return list for more details

#4. A range of outputs: HQ TO HV
print(search_lccn('[HQ TO HV]', 'title', 1)) #can also do this with numbers

#5. A string of call numbers is not allowed to my understanding
#search_lccn('[HJ6799, HJ8001]', 'title,lcc', 2)

#this could be made simpler; right now i return a tuple for simplification when looking up keys in the dictionary
#search_lccn(split_lcc(ex2)[0] + str(split_lcc(ex2)[1]), 'subject')